# K-Fold CV for Boosting Methods Using SKLearn 

In [8]:
#Importing required libraries
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np

from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import randint

from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix, classification_report
from tqdm import tqdm
import wfdb

from sklearn.model_selection import train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression
from numpy import mean
from numpy import absolute
from numpy import sqrt
import pandas as pd
import random

In [9]:
import xgboost as xgb
from sklearn import metrics
from lightgbm import LGBMClassifier

import lightgbm
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer

In [10]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.rcParams['figure.dpi'] = 70 #display 70 dpi in Jupyter Notebook, may consider100 dpi 
plt.rcParams['savefig.dpi'] = 300 #define 300 dpi for saving figures

import seaborn as sns
## here are some settings 
sns.set_style('whitegrid')
sns.set(rc={"figure.dpi":70, 'savefig.dpi':300}) #defining dpi setting
sns.set_context('notebook')
sns.set_style("ticks")
from IPython.display import set_matplotlib_formats

### To load sklearn dataset
sklearn.datasets.load_breast_cancer(*, return_X_y=False, as_frame=False)[source]¶

Load and return the breast cancer wisconsin dataset (classification).

In [11]:
#data = load_breast_cancer(as_frame = True); 
#df = data.frame; 
#print(df.shape)
# In this block use the code we looked at on 6/30 as a reference to save our rr-intervals as a .
rlist = []
records = 'mit-bih-raw\RECORDS' # Replace the _______ with the name of the records file in your MIT-BIH data folder
with open(records) as rfile: #Then we open the file 
                             #The 'with' command only opens the file while we are in it. Automatically closes the file when we're not
    for record in rfile:  # Then we iterate through the lines in the file
        record = record[0:len(record)-1] # Remove any erronious new line characters at the end ('\n')
        rlist.append(record) # Then build an array with it
        

###### Step 1: Initialize all Arrays
             # Below, replace all of the ___ with the command that declares an array/list
             # hint: https://stackoverflow.com/questions/1514553/how-to-declare-an-array-in-python
samples = [] # will house the samples of all subjects
good_list = [] # will list the names of the subjects we successfully extracted
bad_list = [] # will house the names of the subjects we failed to extract
qrs = [] # will house the indices of R-Peaks for all subjects
atr_label = [] # will house the labels for each rhythm annotation for all subjects
atr_locs = [] # will house the locations corresponding to the rhythm annotation labels


###### Step 2: Extract Information
for x in tqdm(rlist): #this will iterate through te records that we found above
  
    try: # A try statement will run the except statement if for some reason the try commands fail
         # In this case I use the try statement because one of the subjects has no signal data causing failure
         # I then use bad_list and good_list so that all of the indices in rlist match with the arrays we initialized in Step 1, above
        ######################################################
            # Below find the wfdb function that will return the information that is described below 
        # Then replace _____ with the correct function call
        samp = wfdb.rdsamp('mit-bih-raw/'+x) # wfdb._____(file_location) will read the signal & header data and return a 2 value array
            # samp[0] - the signal data is the raw reading from the ecg. Each value is a sample taken.
            # samp[1] - the header data includes things about the signal data such as:
              # samples per section, denoted 'fs'
              # number of signals, denoted 'n_sig'
            
        ######################################################
        samples.append(samp) #add it to our array for all subject
            #What is our file extension that has the annotation we want? Find it here and replace _____ with it 
            #hint: READ THE VARIABLE NAMES!!!!
        qrs_tmp = wfdb.rdann('mit-bih-raw/'+x, extension='qrs') #extract the QRS Info
        qrs_locs = np.array(qrs_tmp.sample, dtype='int') #Get just the loccation of R-Peaks from the QRS Info
        qrs.append(qrs_locs) # Add to our array for all subjects
        
            #Do the same thing here
        atr = wfdb.rdann('mit-bih-raw/'+x,extension='atr') #extract the atr info which stores the rhythm type(s) over the whole signal
        atr_label.append(atr.aux_note) # aux_note stores the type of rhythm - main two are '(N' for normal and '(AFIB' for AFIB
        atr_locs.append(np.append(atr.sample, len(samp[0]))) #I add the length of the whole sample to the end for better visualization later
        
        good_list.append(x) # when all extraction is successful append the record name to good_list
    except Exception as exep:
        print(exep) # Alert the user of an exception
        bad_list.append(x) # add to the bad list

        

  0%|          | 0/25 [00:00<?, ?it/s]

sampto must be greater than sampfrom
sampto must be greater than sampfrom


100%|██████████| 25/25 [00:31<00:00,  1.27s/it]


In [12]:
features_df_list = []
for subject in good_list:
    data = pd.read_csv(f'subject-features-dataframes/subject{subject}.csv')
    data = data.assign(Subject = subject)
    #print(data)
    features_df_list.append(data)
#features_maindf = pd.concat(features_df_list, ignore_index = True)

In [13]:
features_maindf = pd.concat(features_df_list, ignore_index = True)
print(features_maindf.columns)

Index(['Unnamed: 0', 'AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility',
       'Rhythm', 'Subject'],
      dtype='object')


In [14]:
def confusion_calcs(confusion_matrix):
    temp_arr = confusion_matrix
    tn = temp_arr[0]
    fp = temp_arr[1]
    fn = temp_arr[2]
    tp = temp_arr[3]
    sensitivity = tp / (tp + fn)
    recall = sensitivity
    specificity = tn / (tn + fp)
    precision = tp / (tp + fp)
    accuracy = (tp+tn) / (tp+tn+fn+fp)
    f1 = 2*precision*recall/(precision+recall)
    return [accuracy, sensitivity, specificity, precision, f1]

In [18]:
# Short to Short

import lightgbm

#Implementing cross validation 
#kf = KFold(n_splits=23, random_state=1, shuffle=True)
model = lightgbm.LGBMClassifier(learning_rate=0.1, n_estimators=100, max_depth=5, random_state=0)

from time import time
import timeit #imports timeit module
start_time = timeit.default_timer() #defines start time so computational time can be calculated

y_values = ['Rhythm']
df = features_maindf



all_features = ['AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility']
dataScores = [[],[],[],[],[],[],[],[],[],[]]
        
for score_idx,temp in tqdm(enumerate(all_features)):    
    
    x_values = [temp]
    print(x_values)

    acc_score = [];
    Truth = [];
    Output = [];


    for idx, subject in enumerate(good_list):

        train_index = ~df['Subject'].str.contains(subject)
        test_index = df['Subject'].str.contains(subject)
        X_train , X_test = df.loc[train_index,x_values], df.loc[test_index,x_values]
        y_train , y_test = df.loc[train_index,y_values].values.ravel(), df.loc[test_index,y_values]

        # Create the LightGBM data containers
        model.fit(X_train,y_train,eval_set=[(X_test,y_test),(X_train,y_train)],
              eval_metric='logloss')

        pred_values = model.predict(X_test)

        acc = accuracy_score(y_test, pred_values)
        acc_score.append(acc)

        Truth.extend(y_test.values.reshape(y_test.shape[0])); ## it is a list
        Output.extend(pred_values); ## it is a list  

        #print(model.feature_importances_)
    #     Importance = pd.DataFrame({'Importance':(model.feature_importances_*100)}, 
    #                           index = (X_train.columns))
    #     Importance.sort_values(by = 'Importance', 
    #                        axis = 0, 
    #                        ascending = True).plot(kind = 'barh', 
    #                                               color = 'r')
    #     plt.xlabel('Variable Importance')
    #     plt.gca().legend_ = None
        #plt.savefig('plot1.png')


    elapsed = timeit.default_timer() - start_time #gives total computation time
    print("---Run time is %s seconds ---" % elapsed) #prints computation time
    print()

    print('Accuracy of each fold: \n {}'.format(acc_score))
    print()
    print('Avg accuracy : \n{}'.format(np.mean(acc_score))); 
    print()
    print('Std of accuracy : \n{}'.format(np.std(acc_score)))

    print(confusion_matrix(Truth, Output))
    print(classification_report(Truth, Output))
    print(acc_score)
    print(elapsed)
    
    temp_array = confusion_matrix(Truth, Output)
    joint = temp_array[0].tolist() + temp_array[1].tolist()
    temp_scores = confusion_calcs(joint)
    
    #dataScores[score_idx].append(all_features[score_idx])
    #dataScores[score_idx].append(acc_score)
    dataScores[score_idx].append(np.mean(acc_score))
    dataScores[score_idx].append(np.std(acc_score))
    
    for confusionScore in temp_scores:
        dataScores[score_idx].append(confusionScore)

0it [00:00, ?it/s]C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


['AppEn']
[1]	training's binary_logloss: 0.622706	valid_0's binary_logloss: 0.570194
[2]	training's binary_logloss: 0.566824	valid_0's binary_logloss: 0.520222
[3]	training's binary_logloss: 0.520392	valid_0's binary_logloss: 0.479057
[4]	training's binary_logloss: 0.481321	valid_0's binary_logloss: 0.444876
[5]	training's binary_logloss: 0.448137	valid_0's binary_logloss: 0.416328
[6]	training's binary_logloss: 0.419735	valid_0's binary_logloss: 0.392324
[7]	training's binary_logloss: 0.395276	valid_0's binary_logloss: 0.37213
[8]	training's binary_logloss: 0.374117	valid_0's binary_logloss: 0.355151
[9]	training's binary_logloss: 0.355741	valid_0's binary_logloss: 0.34081
[10]	training's binary_logloss: 0.339711	valid_0's binary_logloss: 0.328745
[11]	training's binary_logloss: 0.325699	valid_0's binary_logloss: 0.318669
[12]	training's binary_logloss: 0.313423	valid_0's binary_logloss: 0.310249
[13]	training's binary_logloss: 0.302648	valid_0's binary_logloss: 0.303248
[14]	training

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1]	training's binary_logloss: 0.622725	valid_0's binary_logloss: 0.586383
[2]	training's binary_logloss: 0.567504	valid_0's binary_logloss: 0.529042
[3]	training's binary_logloss: 0.521615	valid_0's binary_logloss: 0.48106
[4]	training's binary_logloss: 0.483031	valid_0's binary_logloss: 0.440324
[5]	training's binary_logloss: 0.450253	valid_0's binary_logloss: 0.405577
[6]	training's binary_logloss: 0.422229	valid_0's binary_logloss: 0.375639
[7]	training's binary_logloss: 0.398117	valid_0's binary_logloss: 0.349681
[8]	training's binary_logloss: 0.377259	valid_0's binary_logloss: 0.32715
[9]	training's binary_logloss: 0.35915	valid_0's binary_logloss: 0.307489
[10]	training's binary_logloss: 0.343376	valid_0's binary_logloss: 0.290276
[11]	training's binary_logloss: 0.329594	valid_0's binary_logloss: 0.275077
[12]	training's binary_logloss: 0.317531	valid_0's binary_logloss: 0.261729
[13]	training's binary_logloss: 0.307014	valid_0's binary_logloss: 0.249996
[14]	training's binary_l

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


...

[1]	training's binary_logloss: 0.646677	valid_0's binary_logloss: 0.579489
[2]	training's binary_logloss: 0.610643	valid_0's binary_logloss: 0.536704
[3]	training's binary_logloss: 0.58074	valid_0's binary_logloss: 0.501536
[4]	training's binary_logloss: 0.555686	valid_0's binary_logloss: 0.472235
[5]	training's binary_logloss: 0.534544	valid_0's binary_logloss: 0.447733
[6]	training's binary_logloss: 0.516591	valid_0's binary_logloss: 0.427145
[7]	training's binary_logloss: 0.501275	valid_0's binary_logloss: 0.409568
[8]	training's binary_logloss: 0.488148	valid_0's binary_logloss: 0.394613
[9]	training's binary_logloss: 0.476854	valid_0's binary_logloss: 0.381707
[10]	training's binary_logloss: 0.467111	valid_0's binary_logloss: 0.37069
[11]	training's binary_logloss: 0.458681	valid_0's binary_logloss: 0.361168
[12]	training's binary_logloss: 0.451362	valid_0's binary_logloss: 0.352955
[13]	training's binary_logloss: 0.445004	valid_0's binary_logloss: 0.345724
[14]	training's binary_

C:\Users\NNL\anacondareal\lib\site-packages\sklearn\utils\validation.py:63: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  return f(*args, **kwargs)


[1]	training's binary_logloss: 0.641674	valid_0's binary_logloss: 0.708758
[2]	training's binary_logloss: 0.606602	valid_0's binary_logloss: 0.660159
[3]	training's binary_logloss: 0.577438	valid_0's binary_logloss: 0.618925
[4]	training's binary_logloss: 0.552954	valid_0's binary_logloss: 0.583555
[5]	training's binary_logloss: 0.532274	valid_0's binary_logloss: 0.553079
[6]	training's binary_logloss: 0.514701	valid_0's binary_logloss: 0.526692
[7]	training's binary_logloss: 0.499706	valid_0's binary_logloss: 0.503688
[8]	training's binary_logloss: 0.486852	valid_0's binary_logloss: 0.48362
[9]	training's binary_logloss: 0.475798	valid_0's binary_logloss: 0.465994
[10]	training's binary_logloss: 0.466251	valid_0's binary_logloss: 0.450466
[11]	training's binary_logloss: 0.45798	valid_0's binary_logloss: 0.436748
[12]	training's binary_logloss: 0.450811	valid_0's binary_logloss: 0.424627
[13]	training's binary_logloss: 0.444574	valid_0's binary_logloss: 0.413901
[14]	training's binary_

10it [00:59,  5.91s/it]

[[16570  8173]
 [ 1537 18688]]
              precision    recall  f1-score   support

           0       0.92      0.67      0.77     24743
           1       0.70      0.92      0.79     20225

    accuracy                           0.78     44968
   macro avg       0.81      0.80      0.78     44968
weighted avg       0.82      0.78      0.78     44968

[0.7774599542334096, 0.9103448275862069, 0.8463476070528967, 0.2973621103117506, 0.9164054336468129, 0.8758620689655172, 0.9093881363848669, 0.8502382573179034, 0.8383534136546185, 0.5352811466372657, 0.9718693284936479, 0.6999282124910265, 0.5560090702947846, 0.9929936305732484, 0.47127393838467946, 0.9096145325653522, 0.7936943111720356, 0.927870744373918, 0.36770921386306, 0.879604178119846, 0.9499572284003421, 0.8141870684243565, 0.9403805496828752]
58.919070499999975


In [19]:
export_df = pd.DataFrame(dataScores,columns=['Mean Acc.', 'SD of Acc.', 'Total Acc.', 'Sensitivity', 
                                             'Specificity','Precision','f1'])

export_df = export_df.set_index(pd.Index(['AppEn', 'SampEn', 'CosEn', 'RangeEn', 'FuzzyEn',
       'EntropyAF', 'PermEn', 'BubbleEn', 'MultiscaleEn', 'Volatility']))

In [20]:
export_df

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
AppEn,0.879105,0.162251,0.867839,0.866007,0.869337,0.844178,0.854953
SampEn,0.879496,0.157006,0.870219,0.892608,0.851918,0.831284,0.860855
CosEn,0.878541,0.159536,0.869040,0.889493,0.852322,0.831177,0.859347
RangeEn,0.879287,0.096969,0.878692,0.901310,0.860203,0.840511,0.869849
FuzzyEn,0.770560,0.120493,0.771215,0.825612,0.726751,0.711795,0.764490
EntropyAF,0.860983,0.142201,0.862169,0.907985,0.824718,0.808951,0.855612
PermEn,0.619929,0.116403,0.628158,0.613300,0.640302,0.582238,0.597366
BubbleEn,0.617261,0.103016,0.620530,0.559308,0.670573,0.581205,0.570046
MultiscaleEn,0.853760,0.165901,0.845401,0.853894,0.838459,0.812056,0.832450
Volatility,0.784006,0.195085,0.784069,0.924005,0.669684,0.695730,0.793782


In [21]:
export_df_highlight = export_df.copy()

export_df_highlight.style.highlight_max(color = 'lightgreen', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
AppEn,0.879105,0.162251,0.867839,0.866007,0.869337,0.844178,0.854953
SampEn,0.879496,0.157006,0.870219,0.892608,0.851918,0.831284,0.860855
CosEn,0.878541,0.159536,0.869040,0.889493,0.852322,0.831177,0.859347
RangeEn,0.879287,0.096969,0.878692,0.901310,0.860203,0.840511,0.869849
FuzzyEn,0.770560,0.120493,0.771215,0.825612,0.726751,0.711795,0.764490
EntropyAF,0.860983,0.142201,0.862169,0.907985,0.824718,0.808951,0.855612
PermEn,0.619929,0.116403,0.628158,0.613300,0.640302,0.582238,0.597366
BubbleEn,0.617261,0.103016,0.620530,0.559308,0.670573,0.581205,0.570046
MultiscaleEn,0.853760,0.165901,0.845401,0.853894,0.838459,0.812056,0.832450
Volatility,0.784006,0.195085,0.784069,0.924005,0.669684,0.695730,0.793782


In [23]:
export_df_highlight.style.highlight_min(color = 'lightcoral', axis = 0)

,Mean Acc.,SD of Acc.,Total Acc.,Sensitivity,Specificity,Precision,f1
AppEn,0.879105,0.162251,0.867839,0.866007,0.869337,0.844178,0.854953
SampEn,0.879496,0.157006,0.870219,0.892608,0.851918,0.831284,0.860855
CosEn,0.878541,0.159536,0.869040,0.889493,0.852322,0.831177,0.859347
RangeEn,0.879287,0.096969,0.878692,0.901310,0.860203,0.840511,0.869849
FuzzyEn,0.770560,0.120493,0.771215,0.825612,0.726751,0.711795,0.764490
EntropyAF,0.860983,0.142201,0.862169,0.907985,0.824718,0.808951,0.855612
PermEn,0.619929,0.116403,0.628158,0.613300,0.640302,0.582238,0.597366
BubbleEn,0.617261,0.103016,0.620530,0.559308,0.670573,0.581205,0.570046
MultiscaleEn,0.853760,0.165901,0.845401,0.853894,0.838459,0.812056,0.832450
Volatility,0.784006,0.195085,0.784069,0.924005,0.669684,0.695730,0.793782


In [24]:
export_df.to_csv(f'new-individual-features/LightGBM.csv')